In [2]:
import os
import sys

main_path = os.getcwd()
sys.path.append(main_path)
print("Main path: ", main_path)

Main path:  /home/billy/mental-llm/llm_chatbot_backend


In [ ]:
import torch
from time import sleep
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer

from ai.llm.models.interface.base_ai_model import BaseAiModel
from log import logger


class MentalLlm(BaseAiModel):
    def __init__(self, max_length=512, model_name="MentalLlm", **kwags):
        print("MentalLlm kwags: ", kwags)
        self.max_length = max_length
        self.user_name = "사용자"
        self.device = kwags['device'] if kwags['device'] != None else 'cpu'
        print(f"self.device: {self.device}")

        # 모델과 토크나이저 로드
        load_model_name = "juhoon01/ko_llama3_model_shinhan_2"
        self.model = AutoModelForCausalLM.from_pretrained(
            load_model_name, torch_dtype=torch.float32, device_map=self.device)
        self.tokenizer = AutoTokenizer.from_pretrained(load_model_name)

        logger.info(f"Load {model_name} model complete.")

    def preProcess(self, text: str) -> str:
        """ LLM에 입력하기 전에 전처리를 수행"""
        return text

    def postProcess(self, text: str) -> str:
        """ LLM의 출력을 후처리"""
        text = text['generated_text'].replace("사우", f"{self.user_name}")
        return text

    def generate_response(self, prompt: str, messages=[]):
        """
        prompt(str): 사용자의 입력
            - EX: prompt = "요즘 무기력하고 힘들어요. 에너지가 없어서 아무것도 하기 싫은 기분이 많이 들어요"
        messages(list): 사용자와의 이전 대화 내용
        """
        # LLM에 입력하기 전에 전처리를 수행
        prompt = self.preProcess(prompt)

        pipe = pipeline(task="text-generation", model=self.model,
                        tokenizer=self.tokenizer, max_length=self.max_length)
        result = pipe(f"### 질문: {prompt}\n### 답변:")

        # LLM의 출력을 후처리
        return self.postProcess(result[0])

    def set_max_length(self, max_length):
        self.max_length = max_length


/home/billy/anaconda3/envs/mental-llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-11-10 10:24:16.641244: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 10:24:16.660938: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 10:24:16.660956: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 10

2024-11-10 10:24:17 - log - INFO - 모델 검증을 성공적으로 완료했습니다.


In [4]:
mental_llm = MentalLlm(device='cuda')

MentalLlm kwags:  {'device': 'cuda'}
self.device: cuda


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


PackageNotFoundError: No package metadata was found for bitsandbytes